In [ ]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
df = pd.read_csv("dataset_final.csv")

In [ ]:
def overnight_tweet(hour):
  if hour in range(5) or hour == 23:
    return 1
  else:
    return 0

In [ ]:
df['post_created'] = pd.to_datetime(df['post_created'])
df['hour_of_day'] = df['post_created'].dt.hour
df.head()

,Unnamed: 0,user_id,post_created,followers,friends,statuses,label,cleaned_text,emotion,emotion_encoded,hour_of_day
0,0,14724376,2016-02-20 23:19:00+00:00,350,207,22207,1,bf said heard phone notification going silent ...,fear,4,23
1,1,14724376,2016-02-21 02:44:12+00:00,350,207,22207,1,watched many terrible romantic comedy really h...,sadness,0,2
2,2,14724376,2016-02-21 02:53:24+00:00,350,207,22207,1,maid manhattan still give confusing lady boner...,anger,3,2
3,3,14724376,2016-02-21 02:58:28+00:00,350,207,22207,1,know thats 2002 ralph dang ok,joy,1,2
4,4,14724376,2016-02-21 17:37:56+00:00,350,207,22207,1,threehares tongue tho,joy,1,17


In [ ]:
df['overnight'] = df['hour_of_day'].apply(lambda x: overnight_tweet(x))

In [ ]:
df = df.groupby('user_id').filter(lambda x: 20 <= len(x) <= 1000)

# Verify the changes and show the count of entries per user to confirm
entry_counts = df['user_id'].value_counts()
print(entry_counts)

user_id
1616997456            888
1497350173            792
2285922931            722
1052121847            717
20118423              697
3346224328            650
3045320288            633
18831261              543
60868031              530
763182466098233344    522
3015971504            458
762433972273950725    396
1169875706            360
49548465              352
14724376              341
377696171             331
1306425758            303
468753088             257
1013187241            240
343501537             221
324294391             215
454311273             196
2780518314            193
221788137             188
1629737760            177
1058067937            159
632291359             144
30863895              142
718984718             138
894149342             131
1143982232            125
171999132             121
47622202              120
39248633              119
20863408              117
29053403              112
803100541             109
325328455             100
2369

In [ ]:

# Ensure the 'post_created' column is parsed as datetime
df['post_created'] = pd.to_datetime(df['post_created'])

# Extract the hour of the day from the 'post_created' datetime column
df['hour_of_day'] = df['post_created'].dt.hour

# Determine if the day was on a weekend (Saturday or Sunday)
df['weekend'] = df['post_created'].dt.weekday.apply(lambda x: 1 if x >= 5 else 0)

# Verify the changes and show the first few rows to see the new column
df.head()

In [ ]:
df = df.sort_values(by='post_created')

In [ ]:
columns_to_keep = ['user_id', 'emotion_encoded', 'weekend', 'overnight']

df = df[columns_to_keep]

In [ ]:
df.head()

,user_id,emotion_encoded,weekend,overnight
2674,60868031,1,0,0
2675,60868031,1,0,0
2676,60868031,1,0,0
2677,60868031,1,0,0
2678,60868031,1,0,1


In [ ]:
def generate_row(batch):
    row = []
    for index, row_data in batch.iloc[:-1].iterrows():  # Exclude the last row
        for value in row_data[1:]:
            row.append(value)

    # Add only the "emotion_encoded" value of the last row of the batch
    last_row_emotion_encoded = batch.iloc[-1]['emotion_encoded']
    row.append(last_row_emotion_encoded)

    return row

In [ ]:
def split_group(group):
    num_tweets = len(group)
    num_batches = num_tweets // 10
    batches = []

    for i in range(num_batches):
        start_idx = i * 10
        end_idx = (i + 1) * 10
        batch = group.iloc[start_idx:end_idx]
        batches.append(batch)

    remaining_rows = num_tweets % 10
    if remaining_rows > 0:
        last_batch = group.iloc[num_batches * 10:]
        # Duplicate the last tweet to fill the last batch
        for _ in range(10 - remaining_rows):
            last_batch = pd.concat([last_batch, last_batch.iloc[[-1]]])
        batches.append(last_batch)

    return batches

In [ ]:
new_df = pd.DataFrame(columns=range(28))

for user_id, group in df.groupby('user_id'):
    batches = split_group(group)

    for batch in batches:
        row = generate_row(batch)
        if(len(row) == 28):
          new_df.loc[len(new_df)] = row

In [ ]:
new_df.to_csv("filtered_input_matrix.csv", index=False)